<a href="https://colab.research.google.com/github/aashmauprety/Final_UAV/blob/master/TF_Agent_UAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd TF-Agent/

/content/TF-Agent


In [0]:
from enum import Enum
import numpy as np

In [0]:
class ActionResult(Enum):
  COLLISION = 1
  NO_MOVE = 2
  SUCCESS = 3

In [0]:
#MY CUSTOM ENVIRONMENT 
class UAV():
  #initialize
  def __init__(self):
    self._state = np.zeros(36, dtype= int8)
    self._state[5] =1 #initial position of UAV1
    self._state[9] = 2 #initial popsition of UAV2
    self._state[15] = 3 #position of station which is stationary
    self._game_ended = False


  #Reset the environement to its initial state.
  def reset(self):
    self._state = np.zeros(36, dtype = int32)
    self._state[5] = 1
    self._state[9] = 2
    self._state[15] = 3
    self._game_ended = False

  def __is_success(self, position1, position2):
    return position1 == 15 or position2 == 15

  def move_uav(self, current_state1, current_state2, next_state1, next_state2):

    #check if success occured 
    if self._is_success(next_state1, next_state2):
      self._state[current_state1] = 0
      self._state[next_state1] = 1
      self._state[current_state2] = 0
      self._state[next_state2] = 2
      self._game_ended = True

      return ActionResult.SUCCESS

    if   






      
